In [174]:
import pandas as pd
import numpy as np

In [175]:
df = pd.read_csv('data/Acidentes/sinistros_com_chuva_2022-2025.csv', delimiter=';')

In [ ]:
colunas_desejadas = [
    'data_sinistro', 'hora_sinistro', 'tipo_via', 'latitude', 'longitude', 'tp_veiculo_bicicleta', 'tp_veiculo_caminhao', 
    'tp_veiculo_motocicleta', 'tp_veiculo_nao_disponivel', 'tp_veiculo_onibus', 
    'tp_veiculo_outros', 'tp_veiculo_automovel', 'Chuva'
]

df = df[colunas_desejadas]

df['Chuva'] = df['Chuva'].apply(lambda x: 0 if x == 'Sem chuva' else 1)
df['Sinistro'] = 1

df['hora_sinistro'] = df['hora_sinistro'].apply(lambda x: '00:00' if x == '99:99' else x)
df['hora_sinistro'] = pd.to_datetime(df['hora_sinistro'], format='%H:%M').dt.hour


tipo_via_map = {
    'VIAS MUNICIPAIS': 0,
    'RODOVIAS': 1,
    'NAO DISPONIVEL': 2
}


df['tipo_via'] = df['tipo_via'].map(lambda x: tipo_via_map.get(x, -1))  

df["data_sinistro"] = pd.to_datetime(df["data_sinistro"], dayfirst=True, errors="coerce")
df["mes"] = df["data_sinistro"].dt.month
df["dia_semana"] = df["data_sinistro"].dt.weekday  
df

,data_sinistro,hora_sinistro,tipo_via,latitude,longitude,tp_veiculo_bicicleta,tp_veiculo_caminhao,tp_veiculo_motocicleta,tp_veiculo_nao_disponivel,tp_veiculo_onibus,tp_veiculo_outros,tp_veiculo_automovel,Chuva,Sinistro,mes,dia_semana
0,2022-01-01,4,0,"-22,31817024","-49,11414784",0,0,0,0,0,0,0,0,1,1,5
1,2022-01-01,4,0,"-22,30523809","-49,09794557",0,0,0,0,0,0,0,0,1,1,5
2,2022-01-01,4,0,"-22,30645978","-49,10445329",0,0,1,0,0,0,0,0,1,1,5
3,2022-01-02,19,0,"-22,31234912","-49,12226505",0,0,0,0,0,0,0,0,1,1,6
4,2022-01-02,20,0,"-22,3381656","-49,05324406",0,0,1,0,0,0,1,0,1,1,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8184,2025-02-28,22,0,"-22,391376","-49,069106",0,0,0,0,0,0,3,0,1,2,4
8185,2025-02-28,13,0,"-22,2957081","-49,0455861",0,0,1,0,0,0,1,0,1,2,4
8186,2025-02-28,13,0,"-22,3385513","-49,06961657",0,0,0,0,0,0,2,0,1,2,4
8187,2025-02-28,18,0,"-22,3057032","-49,0519483",0,0,0,0,0,0,1,0,1,2,4


In [177]:
df['latitude'] = df['latitude'].str.replace(',', '.', regex=False).astype(float)
df['longitude'] = df['longitude'].str.replace(',', '.', regex=False).astype(float)

In [178]:
df.head()

,data_sinistro,hora_sinistro,tipo_via,latitude,longitude,tp_veiculo_bicicleta,tp_veiculo_caminhao,tp_veiculo_motocicleta,tp_veiculo_nao_disponivel,tp_veiculo_onibus,tp_veiculo_outros,tp_veiculo_automovel,Chuva,Sinistro,mes,dia_semana
0,2022-01-01,4,0,-22.318170,-49.114148,0,0,0,0,0,0,0,0,1,1,5
1,2022-01-01,4,0,-22.305238,-49.097946,0,0,0,0,0,0,0,0,1,1,5
2,2022-01-01,4,0,-22.306460,-49.104453,0,0,1,0,0,0,0,0,1,1,5
3,2022-01-02,19,0,-22.312349,-49.122265,0,0,0,0,0,0,0,0,1,1,6
4,2022-01-02,20,0,-22.338166,-49.053244,0,0,1,0,0,0,1,0,1,1,6


In [ ]:
# Função para gerar pequenas variações nas coordenadas (latitude/longitude)
def gerar_perturbacao(coordenada, magnitude=0.0001):
    """
    Aplica uma pequena variação nas coordenadas para gerar novas amostras
    sem distorcer a localização.
    """
    return coordenada + np.random.uniform(-magnitude, magnitude)

num_acidentes = len(df)  # Número de acidentes reais
num_nao_acidentes = num_acidentes * 2 #Proporção de 1:2


df_negativo = df.copy()  
df_negativo['Sinistro'] = 0  

df_negativo['latitude'] = df_negativo['latitude'].apply(gerar_perturbacao)
df_negativo['longitude'] = df_negativo['longitude'].apply(gerar_perturbacao)

df_negativo['tipo_via'] = df['tipo_via']  # Manter o tipo de via original


df_negativo_amostras = df_negativo.sample(n=num_nao_acidentes, replace=True, random_state=42)

In [180]:
df_final = pd.concat([df, df_negativo_amostras], ignore_index=True)
df_final = df_final.sample(frac=1).reset_index(drop=True)
df_final = df_final.drop(columns=["data_sinistro"])
df_final.head()

,hora_sinistro,tipo_via,latitude,longitude,tp_veiculo_bicicleta,tp_veiculo_caminhao,tp_veiculo_motocicleta,tp_veiculo_nao_disponivel,tp_veiculo_onibus,tp_veiculo_outros,tp_veiculo_automovel,Chuva,Sinistro,mes,dia_semana
0,5,0,-22.320106,-49.068455,0,0,1,0,0,0,1,0,0,4,0
1,21,0,-22.351964,-49.060317,0,0,0,0,0,0,1,0,0,8,3
2,22,0,-22.348451,-49.040764,0,0,0,0,0,0,0,0,0,1,2
3,16,0,-22.317594,-49.109013,0,0,0,0,0,0,2,0,0,1,2
4,15,0,-22.317187,-49.075325,0,0,0,0,0,0,0,0,1,7,1


In [181]:
df_final.to_csv('df_amostragem_negativa.csv', index=False)